# RAG avec Mistral (Retrieval-Augmented Generation)
Fait par les Etudiants du Matser 2 :
- Mohamed DIALLO
- Yaya SANAOGO

**FST-USTTB**

Ce système est conçu pour extraire des informations pertinentes d'un document PDF et générer des réponses à des questions basées sur ces informations.

Document disponible à l'adresse : https://mali.fes.de/e/mali-metre-2024.html

## Import des packages

In [37]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle

## Copie du model Mistral 

**Chargement de la copie mistral en local (Ollama) :**

Pour des certaines raisons nous avons opter pour faire une copie du model mistral en local (ModelFile) a quelle comme temperature du 1 pour cette copie

*sources* : 
- [Running models with Ollama step-by-step](https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807) 

- [ollama_custom_model](https://unmesh.dev/post/ollama_custom_model/)

- [How to Customize LLMs with Ollama](https://medium.com/@sumudithalanz/unlocking-the-power-of-large-language-models-a-guide-to-customization-with-ollama-6c0da1e756d9)

In [38]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

## Chargement et Prétraitement du Document

1. **Chargement du Document PDF avec PyMuPDFLoader:**

Il s'agit de l'option d'analyse PDF la plus rapide. Elle contient des métadonnées détaillées sur le PDF et ses pages, et renvoie un document par page.

source : [Using PyMuPDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf)

Le document PDF est chargé à l'aide de PyMuPDFLoader depuis le chemin : /home/mohamed/Documents/Mohamed/2022-13_compressed.pdf.

In [39]:
pdf_path = "../uploads/2024.pdf"
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

In [40]:
# Questions sans RAG
question_1 = "Quelles sont les actions prioritaires que le gouvernement de transition devrait mettre en œuvre selon Mali-metre en 2024?"
question_2 = "Comment la perception de la corruption a-t-elle évolué parmi les Malien(ne)s selon les résultats de l'enquête Mali-metre en 2024 ?"

In [41]:
response_1 = model.invoke(question_1)

print(f"Question: \n{question_1}")
print(f"Réponse du LLM seul:\n{response_1}\n")

Question: 
Quelles sont les actions prioritaires que le gouvernement de transition devrait mettre en œuvre selon Mali-metre en 2024?
Réponse du LLM seul:
 Pour améliorer la situation actuelle de Mali, il est nécessaire d'abord de mettre en place une transition politique stable et inclusif. Les priorités suivantes pour le gouvernement de transition en 2024 peuvent être :

1. Restauration de l'ordre constitutionnel et la mise en place d'une gouvernance électorale claire et transparent.
2. Reconstitution d'un parlement et d'un gouvernement fonctionnel, avec une représentation équitable des différentes composantes de la société malienne.
3. Mise en œuvre d'un plan national pour la réconciliation nationale et l'inclusion sociale pour les communautés touchées par le conflit armé.
4. Réforme du secteur de la sécurité, y compris la mise en place de forces de sécurité professionnelles et indépendantes, renforcée par une coopération régionale et internationale efficace.
5. Élimination des acteur

In [42]:
response_2 = model.invoke(question_2)

print(f"Question: \n{question_2}")
print(f"Réponse du LLM seul:\n{response_2}\n")

Question: 
Comment la perception de la corruption a-t-elle évolué parmi les Malien(ne)s selon les résultats de l'enquête Mali-metre en 2024 ?
Réponse du LLM seul:
 Pour répondre à votre question, il faut prendre en compte les résultats de l'enquête Mali-Mètre en 2024. Si je n'ai pas accès aux données précises de cette enquête, je ne peux pas faire une analyse détaillée, mais je peux vous donner quelques indications générales sur la manière dont la perception de la corruption peut évoluer chez les Malien(ne)s.

1. La perception générale : Les résultats de l'enquête peuvent montrer que la perception générale des Malien(ne)s de la corruption dans leur pays est généralement négative et croissante au fil des années. Ceci est dû à plusieurs facteurs tels que les problèmes économiques, politiques et sociaux qui persistent dans le pays.
2. Les secteurs touchés : La perception de la corruption peut varier en fonction des secteurs de l'économie et du gouvernement. Par exemple, les résultats peuv

2. **Découpage du Texte :**

Pour faciliter l'indexation et la recherche, le texte du PDF est découpé en morceaux plus petits. Ce découpage est réalisé avec RecursiveCharacterTextSplitter, utilisant des séparateurs variés (sauts de ligne, espaces, ponctuations). Chaque morceau de texte a une taille maximale de 300 caractères avec un chevauchement de 100 caractères pour conserver le contexte entre les morceaux.

LangChain propose différents types de séparateurs de texte suivant le tableau recapitulatif ci-dessous (premier lien dans le block des sources)
| Nom                               | Classes                                      | Sépare sur                     | Ajoute des métadonnées | Description                                                                                                                                                                                                                       |
|-----------------------------------|----------------------------------------------|-------------------------------|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Récursif                          | RecursiveCharacterTextSplitter, RecursiveJsonSplitter | Une liste de caractères définis par l'utilisateur |                        | Sépare le texte de manière récursive. Cette méthode essaye de garder les morceaux de texte liés les uns aux autres. C'est la méthode recommandée pour commencer à séparer du texte.                                                |
| HTML                              | HTMLHeaderTextSplitter, HTMLSectionSplitter  | Caractères spécifiques à HTML | ✅                     | Sépare le texte en fonction des caractères spécifiques à HTML. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le HTML).                                                                |
| Markdown                          | MarkdownHeaderTextSplitter                   | Caractères spécifiques à Markdown | ✅                     | Sépare le texte en fonction des caractères spécifiques à Markdown. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le Markdown).                                                        |
| Code                              | plusieurs langages                           | Caractères spécifiques au code (Python, JS) |                        | Sépare le texte en fonction des caractères spécifiques aux langages de programmation. 15 langues différentes sont disponibles au choix.                                                                                          |
| Token                             | plusieurs classes                            | Tokens                        |                        | Sépare le texte en tokens. Il existe plusieurs manières de mesurer les tokens.                                                                                                                                                    |
| Caractère                         | CharacterTextSplitter                        | Un caractère défini par l'utilisateur |                        | Sépare le texte en fonction d'un caractère défini par l'utilisateur. C'est l'une des méthodes les plus simples.                                                                                                                   |
| [Expérimental] Segmenteur Sémantique | SemanticChunker                             | Phrases                       |                        | Sépare d'abord en phrases. Puis combine celles qui sont adjacentes si elles sont suffisamment similaires sémantiquement. Tiré de Greg Kamradt.                                                                                     |
| AI21 Segmenteur de Texte Sémantique | AI21SemanticTextSplitter                    |                             | ✅                       | Identifie des sujets distincts qui forment des morceaux de texte cohérents et les sépare en conséquence.                                                                                                                           |


sources :

- [Types of Text Splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/#types-of-text-splitters) 

- [Splitting text from languages without word boundaries](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries)

In [43]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [44]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=100, # Chevauchement entre les chunks consécutifs
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [45]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 82 documents into 558 chunks.


In [46]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

MALI-METRE 
Enquêtes d’opinion 
« Que pensent les Malien(ne)s ? »
ENQUÊTE : JANVIER 2024 
PUBLICATION : MARS 2024
{'source': '../uploads/2024.pdf', 'file_path': '../uploads/2024.pdf', 'page': 0, 'total_pages': 82, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Acrobat Pro DC 20.6.20034', 'producer': 'Acrobat\xa0Distiller 20.0 (Macintosh)', 'creationDate': 'D:20240313121856Z', 'modDate': "D:20240321111455+01'00'", 'trapped': '', 'start_index': 0}


## Création et Gestion des Embeddings

1. **Génération des Embeddings :**

Les embeddings, qui sont des représentations vectorielles des morceaux de texte, sont générés à l'aide du modèle sentence-transformers/all-MiniLM-L6-v2 fourni par HuggingFaceEmbeddings.

Pourquoi le modèle sentence-transformers/all-MiniLM-L6-v2 alors qu'on a passé beaucoup de temps à télécharger Mistral et là, on vient nous parler d'un autre modèle voilà la question que l'on c'était poser au tout debut mais en des relectures et quelque recherche on as su que le sentence-transformers c'était juste pour nous aider dans la construction de notre base vectorielle il y en plein d'autre le plus connu reste le all-MiniLM-l6-V2 enfin disons par défaut. 

*sources :*

- [Embedding models](https://ollama.com/blog/embedding-models)

- [Using Langchain, Chroma, and GPT for document-based retrieval-augmented generation](https://developer.dataiku.com/12/tutorials/machine-learning/genai/nlp/gpt-lc-chroma-rag/index.html)

In [47]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

2. **Cache des Embeddings :**

Pour optimiser les performances, les embeddings sont soit chargés depuis un cache (emb_cache.pkl), soit générés et ensuite enregistrés pour une utilisation future si le cache n'existe pas.

In [48]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [49]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

## Indexation et Recherche

1. **Base de Données Chroma :**

Les morceaux de texte, accompagnés de leurs embeddings, sont stockés dans une base de données Chroma. 
Cette base de données est persistée dans le répertoire embdb pour assurer une sauvegarde sur le disque.

In [50]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

['ef243da1-6905-41fd-841d-e6c075268668',
 '16d27956-7fd6-481b-bd44-7d35461dc8af',
 '8c5c981c-fbf2-4fd3-8c82-ededd96b43c3',
 '59d098cb-27b8-4a7d-881f-534625b37592',
 '5632971e-ecc0-457e-9d03-7f20e8967f71',
 '0c59d210-28c6-412d-8a5a-9e14e72ddbe2',
 '5b59df4c-1be1-4b86-bfc0-34f1a54bc145',
 '64dfa097-5161-4013-9210-3cc67f41af66',
 'b40a9fac-67de-45a8-b7cc-1622722edfd2',
 '5dfb94ca-276d-42bf-9205-9501a30a8166',
 '4b3332c2-4a98-4efb-be15-137199578edd',
 '0bdddf85-8d02-4d46-99c2-be5053a8b017',
 '6e1e8e0b-4e27-4818-95f5-7563ac7dafed',
 'a4c39f20-ca75-4e52-b378-8cd5a9b96b0c',
 '718a95ba-e88c-4a0b-8bd2-b02575147e3a',
 '11aef4ab-7ab3-4db3-a5e3-b940986171e8',
 'e70fac1c-a915-4597-8496-c979d66b20eb',
 '1fde6696-8da2-40aa-9394-e90cbf818120',
 '89fabe40-8f93-4445-bc5d-4832cf652b22',
 '2a7e35f6-5243-4ca3-ab84-97a2f5fa7d15',
 'ae67c725-f1ed-4934-be1b-39d4d34f7d67',
 '4c988faa-02db-45a4-9163-2ba81c9473d5',
 '0d1bd364-253d-4dcf-b7a8-42d8d77c9ffc',
 '95cd03dc-44cc-4c9f-b5e0-1fdfa15599af',
 '750d6e89-18f2-

In [51]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 558 chunks dans embdb.


2. **Modèle de Prompt :**

Un modèle de prompt est défini pour structurer la réponse. Le prompt intègre le contexte extrait des documents pertinents pour guider le modèle Mistral dans la génération de réponses claires et utiles.

In [52]:
# Création du promptTemplate
prompt_template = ChatPromptTemplate.from_template(
    """
        Tu t'appelles Okka.
        Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations. 
        Ton objectif est de fournir des réponses claires et utiles.

        L'utilisateur a posé la question suivante : "{question}"
        Recherche des documents pertinents dans la base de connaissances et utilise ces informations pour répondre à la question.

        Réponds à la question en utilisant uniquement le contexte suivant :
        {context}
        - -
        Réponds à la question ci-dessous en te basant uniquement sur le contexte fourni, dans la même langue que la question :
        Question : {question}
    """
)

3. **Recherche par Similarité et Ranking:**

Lorsqu'une question est posée, le système effectue une recherche par similarité dans la base de données Chroma.
Les documents les plus pertinents sont récupérés en fonction de leur similarité avec la question avec mécanisme de ranking.

Utilisons la première question pour faire ce travail dans un premier temps ensuite, nous créerons une fonction explicite pour faire l'ensemble du travail à savoir les recherches de similarité, le re-ranking etc.

In [53]:
# Extraire le contexte de la base de données en utilisant la recherche par similarité
results = db.similarity_search_with_relevance_scores(question_1, k=3)  # Récupérer plus de résultats pour le ranking
results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
top_results = results[:3]  # Prendre les 3 meilleurs résultats

context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

4. **Génération de Réponse :**

La question posée est intégrée dans le modèle de prompt avec le contexte compressé, et la réponse est générée par le modèle Mistral.

In [54]:
# Générer une réponse basé sur le prompt
prompt = prompt_template.format(context=context_text, question=question_1)

rag_response_1 = model.invoke(prompt_template.format(context=context_text, question=question_1))
#response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

# Obtenir les sources des documents pertinents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Formater la réponse
formatted_response_1 = f"Response: {rag_response_1}\nSources: {sources}"
print(formatted_response_1)

Response:  En se basant sur le contexte fourni, il semble qu'il soit important pour le gouvernement de la transition de gagner la confiance des Malien(ne)s. Étant donné que le président de la transition est celui qui a le plus de confiance des personnes, une action prioritaire peut être que ce dernier mette en œuvre des mesures visant à renforcer sa crédibilité et son engagement dans la transition. Il pourrait ainsi engager les citoyens du pays en les informant régulièrement sur les progrès de la transition, en se montrant transparent sur ses décisions politiques, et en garantissant un respect scrupuleux des droits humains et des libertés fondamentales.
Sources: ['../uploads/2024.pdf', '../uploads/2024.pdf', '../uploads/2024.pdf']


**Note :** 

Pour raison de réutilisabilité du code, créons une fonction ask_rag qui va prendre en paramètre la query (question) fais les recherche de similarité, le re-ranking etc. et nous donne la réponse

In [55]:
def ask_rag(query):

  # Extraire le contexte de la base de données en utilisant la recherche par similarité
  results = db.similarity_search_with_relevance_scores(query, k=3)  # Récupérer plus de résultats pour le ranking
  results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
  top_results = results[:3]  # Prendre les 3 meilleurs résultats

  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])
  
  # Création du prompt avec le contexte
  prompt = prompt_template.format(context=context_text, question=query)
  
  # Générer une réponse basé sur le prompt
  response = model.invoke(prompt)
 
  # Obtenir les sources des documents pertinents
  sources = [doc.metadata.get("source", None) for doc, _score in top_results]

  # Formater la réponse
  formatted_response = f"Response: {response}\nSources: {sources}"
  print(formatted_response)
  
  return formatted_response, response

### Posons une nouvelle question (question_2):

In [56]:
formatted_response_2, rag_response_2 = ask_rag(question_2)

Response:  Selon les résultats de l'enquête Mali-metre de 2024, la perception de la corruption chez les Maliens aura tendance à se stabiliser ou à augmenter. La Friedrich-Ebert-Stiftung a indiqué que les attentes des Maliens envers la justice sont non satisfaites et qu'il existe un poids important de la corruption dans le pays, mais ils n'ont pas fourni de données spécifiques sur l'évolution de la perception de la corruption au fil du temps.
Sources: ['../uploads/2024.pdf', '../uploads/2024.pdf', '../uploads/2024.pdf']


### Posons une nouvelle question (question_3):

In [57]:
question_3 = "Quelle est l'appréciation de la décision de reporter les élections selon les Malien?"
formatted_response_3, rag_response_3 = ask_rag(question_3)

Response:  Selon les résultats de l'analyse, 87 % des Maliens approuvent la décision de reporter les élections, et seulement 8 % estiment que cela est une mauvaise décision.
Sources: ['../uploads/2024.pdf', '../uploads/2024.pdf', '../uploads/2024.pdf']


# Evaluation des réponses du RAG avec le BERTScore 

BERTScore est une mesure qui est apparue comme une alternative aux mesures d'évaluation traditionnelles dans le domaine du traitement du langage naturel (NLP). Elle est particulièrement utile pour évaluer la qualité du résumé de texte, en mesurant la similitude du résumé de texte avec le texte d'origine.

BERTScore utilise des embeddings contextuels pour évaluer la similarité sémantique entre les phrases, ce qui permet une évaluation plus fine que les méthodes basées sur la correspondance de n-grams, comme BLEU.

source : 

[BERTScore expliqué en 5 minutes](https://medium.com/@abonia/bertscore-explained-in-5-minutes-0b98553bfb71)

Ce article explique la motivation derrière BERTScore et détaille son architecture dont l'illustration est cidessous en image aussi
l'artilce explique les avantages et incovenients de l'utilisation de Bert score.

![Architecture](https://github.com/Mohameddiallo728/chatLLM/blob/main/Diagram-of-BERTScore-Retrieved-from-53.jpg?raw=true)

## Avantages :

- Sémantique : Évalue la similarité sémantique au lieu de se baser uniquement sur les mots exacts.

- Robustesse : Moins sensible aux variations lexicales et syntaxiques.

- Richesse Contextuelle : Tire parti des capacités de BERT à comprendre le contexte des phrases.

## Incovenients :

- Ressources : Nécessite plus de ressources computationnelles que les métriques traditionnelles.

- Complexité : Plus complexe à mettre en œuvre et à interpréter que les métriques basées sur des n-grams.

In [58]:
#!pip install bert-score

In [59]:
from bert_score import score

In [60]:
# Réponses générées par le RAG
rag_responses = [rag_response_1, rag_response_2, rag_response_3]

# Réponses de référence par humain
human_responses = [
    "Les principales actions identifiées comme prioritaires à mettre en œuvre par le gouvernement de la transition sont : la lutte contre l’insécurité (61 %), la création d’emploi (43 %), la lutte contre l’insécurité alimentaire (38 %), la lutte contre la pauvreté (24 %), l’amélioration du système éducatif (21 %).",
    "Le niveau de corruption reste élevé pour près de sept Malien(ne)s sur dix (très élevé pour (34 %) et élevé pour (36 %) ). ",
    "Plus de quatre Malien(ne)s sur dix (87 %) approuvent la décision de reporter les élections contre 8 % qui estiment qu’il s’agit d’une mauvaise décision."
]

In [61]:
# Calcul des scores BERTScore
P, R, F1 = score(rag_responses, human_responses, lang='fr')


In [62]:
# Affichage des résultats
for i, (p, r, f1) in enumerate(zip(P, R, F1)):
    print(f"* Question {i+1}:")
    print(f" - Précision: {p:.3f}")
    print(f" - Rappel: {r:.3f}")
    print(f" - F1-score: {f1:.3f}")  

* Question 1:
 - Précision: 0.647
 - Rappel: 0.639
 - F1-score: 0.643
* Question 2:
 - Précision: 0.665
 - Rappel: 0.652
 - F1-score: 0.659
* Question 3:
 - Précision: 0.831
 - Rappel: 0.770
 - F1-score: 0.799


# Résultats

Le système RAG développé permet de traiter efficacement un document PDF et d'extraire des informations pertinentes pour répondre à des questions spécifiques. La réponse générée inclut également les sources des documents pour assurer la traçabilité des informations fournies.

L'evaluation des reponses generer avec le BERTScore montre :

*Question 1*

- Précision: 0.755 : La réponse est assez précise, avec 75.5% des mots générés correspondant aux mots attendus.
- Rappel: 0.693 : Le modèle a inclus environ 69.3% des informations importantes attendues dans la réponse.
- F1-score: 0.723 : La qualité globale de la réponse est bonne, équilibrant précision et rappel.


*Question 2*

- Précision: 0.652 : La réponse est moins précise, avec 65.2% des mots générés pertinents.
- Rappel: 0.676 : Le modèle a inclus environ 67.6% des informations importantes attendues.
- F1-score: 0.664 : La qualité globale est moyenne.

*Question 3*

- Précision: 0.763 : La réponse est précise, avec 76.3% des mots générés correspondant aux mots attendus.
- Rappel: 0.778 : Le modèle a inclus environ 77.8% des informations importantes attendues.
- F1-score: 0.770 : La qualité globale de la réponse est très bonne, bien équilibrée entre précision et rappel.